# Visualization - Relative Performance App

Dietterich-inspired visuals. Performance relative to the 0% one.

Now also with better sliders for different kinds of analysis.

## Preliminaries

### Imports

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import json
import sys
import pickle as pkl
import warnings

from os.path import dirname

# Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State

In [2]:
# Custom

root_dir = dirname(dirname(os.getcwd()))
src_dir = os.path.join(root_dir, 'src')
sys.path.append(src_dir)

import exp
from exp.utils.extra import mem_usage
from exp.runner.RunExp import RunExp
from exp.runner.RunMercs import RunMercs
from exp.eval.preprocess import *
from exp.eval.preprocess import _insert_inf_time_base,_insert_inf_time_relative
from exp.visual.menus import (generate_dropdown_menu,
                              generate_dropdown_menus_from_df,
                              generate_slider_menu)
from exp.visual.plots import (generate_graph)
from exp.visual.callback import (extract_menu_inputs_menu_names_from_layout,
                                filter_dataframe)

In [3]:
root_dir

'/cw/dtailocal/Repos/random-walks-random-forests'

### Methods

Some custom methods I need in this notebook.

In [4]:
def merge_aggregated_outputs_multiple_exps(exp_idxs, new_root, **kwargs):
    """
    Merge aggregated outputs from multiple experiments.
    """
    
    f = collect_aggregated_outputs_from_exp
    
    gen = (f(exp_idx, new_root, **kwargs) for exp_idx in exp_idxs)
    
    result = {}
    for g in gen:
        result = {k: pd.concat([result.get(k, None),v], sort=False)
                  for k,v in g.items()}    
        
    return result

def collect_aggregated_outputs_from_exp(exp_idx, new_root, **kwargs):
    """
    Load the aggregated outputs by a single experiment.
    """
    
    # Preliminaries
    dfs = {}
    
    # Actions
    re = RunExp.load(idx=exp_idx, **kwargs)
    for output in re.aggr_outputs:
        dfs[output] = re.load_output(kind=output, new_root=new_root)
    return dfs

## Global Parameters

This is the single most important thing you need to specify, i.e., from which experiments do you want to collect the results?

In [5]:
exp_idxs = [1]
#exp_idxs = [15, 100]

## Collect Data

Now, the actual work starts.

In [6]:
dfs = merge_aggregated_outputs_multiple_exps(exp_idxs, new_root=root_dir)

In [7]:
dfs.keys()

dict_keys(['results', 'timings', 'mod_config', 'qry_codes'])

In [8]:
dfs['mod_config'].head()

,,dataset,predict.algo,predict.its,predict.param,fit.sel.param,fit.ind.max_depth,fit.ind.type,fit.sel.its,mod.type,mod.keyword
idx,f_idx,,,,,,,,,,
2,0,nltcs,MI,0.1,0.95,2,4,DT,4,Mercs,md4
3,0,msnbc,MI,0.1,0.95,2,4,DT,4,Mercs,md4
4,0,nltcs,MAFI,0.1,0.95,2,4,DT,4,Mercs,md4
5,0,msnbc,MAFI,0.1,0.95,2,4,DT,4,Mercs,md4


In [9]:
name_contains = ('predict.algo', 'predict.its', 'predict.param')

In [10]:
df_res = preprocess_aggr_df(dfs['results'], kind='res')
df_qry = preprocess_aggr_df(dfs['qry_codes'], kind='qry')
df_cfg = preprocess_aggr_df(dfs['mod_config'],
                            kind='cfg',
                            include_columns=name_contains)
df_tmg = preprocess_aggr_df(dfs['timings'], kind='tmg')

In [11]:
df_cfg.head()

,dataset,predict.algo,predict.its,predict.param,fit.sel.param,fit.ind.max_depth,fit.ind.type,fit.sel.its,mod.type,mod.keyword,name
idx,,,,,,,,,,,
2,nltcs,MI,0.1,0.95,2,4,DT,4,Mercs,md4,predict.algo=MI|predict.its=0.1|predict.param=...
3,msnbc,MI,0.1,0.95,2,4,DT,4,Mercs,md4,predict.algo=MI|predict.its=0.1|predict.param=...
4,nltcs,MAFI,0.1,0.95,2,4,DT,4,Mercs,md4,predict.algo=MAFI|predict.its=0.1|predict.para...
5,msnbc,MAFI,0.1,0.95,2,4,DT,4,Mercs,md4,predict.algo=MAFI|predict.its=0.1|predict.para...


In [12]:
df_tmg.head()

ind_time  inf_time
idx q_idx                    
2   0      1.845375  0.008549
    1      1.845375  0.007563
    2      1.845375  0.008263
    3      1.845375  0.008458
    4      1.845375  0.009107

In [13]:
df_cfg_f = df_cfg[['dataset', 'predict.algo', 'predict.its', 'predict.param']]
join_idx_names = ['idx']
df_1 = df_tmg.join(df_cfg_f, how='inner', on=join_idx_names)
#df_1.head()

In [14]:
df_1.head()

ind_time  inf_time dataset predict.algo  predict.its  predict.param
idx q_idx                                                                     
2   0      1.845375  0.008549   nltcs           MI          0.1           0.95
    1      1.845375  0.007563   nltcs           MI          0.1           0.95
    2      1.845375  0.008263   nltcs           MI          0.1           0.95
    3      1.845375  0.008458   nltcs           MI          0.1           0.95
    4      1.845375  0.009107   nltcs           MI          0.1           0.95

In [15]:
df_2 = _insert_inf_time_base(df_1, baseline=('predict.algo', 'MI'))
df_2.head()

ind_time  inf_time dataset predict.algo  predict.its  \
idx q_idx                                                         
2   0      1.845375  0.008549   nltcs           MI          0.1   
    1      1.845375  0.007563   nltcs           MI          0.1   
    2      1.845375  0.008263   nltcs           MI          0.1   
    3      1.845375  0.008458   nltcs           MI          0.1   
    4      1.845375  0.009107   nltcs           MI          0.1   

           predict.param  inf_time_base  
idx q_idx                                
2   0               0.95       0.008549  
    1               0.95       0.007563  
    2               0.95       0.008263  
    3               0.95       0.008458  
    4               0.95       0.009107

In [16]:
df_3 = _insert_inf_time_relative(df_2)
df_3.drop(columns=['dataset'], inplace=True)
df_3.head()

ind_time  inf_time predict.algo  predict.its  predict.param  \
idx q_idx                                                                
2   0      1.845375  0.008549           MI          0.1           0.95   
    1      1.845375  0.007563           MI          0.1           0.95   
    2      1.845375  0.008263           MI          0.1           0.95   
    3      1.845375  0.008458           MI          0.1           0.95   
    4      1.845375  0.009107           MI          0.1           0.95   

           inf_time_base  inf_time_rel  
idx q_idx                               
2   0           0.008549           1.0  
    1           0.007563           1.0  
    2           0.008263           1.0  
    3           0.008458           1.0  
    4           0.009107           1.0

In [17]:
df_plt = build_df_plt(df_res, df_qry, df_cfg, df_3)
mem_usage(df_plt)
df_plt.head()


    35.27 kiloB
    


score  \
idx name                                               q_idx             
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0      0.805131   
                                                       1      0.795946   
                                                       2      0.792489   
                                                       3      0.494750   
                                                       4      0.328801   

                                                              ind_time  \
idx name                                               q_idx             
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0      1.845375   
                                                       1      1.845375   
                                                       2      1.845375   
                                                       3      1.845375   
                                                       4      1.845375   

                                                              inf_time  \
idx name                                               q_idx             
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0      0.008549   
                                                       1      0.007563   
                                                       2      0.008263   
                                                       3      0.008458   
                                                       4      0.009107   

                                                             predict.algo  \
idx name                                               q_idx                
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0               MI   
                                                       1               MI   
                                                       2               MI   
                                                       3               MI   
                                                       4               MI   

                                                              predict.its  \
idx name                                               q_idx                
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0              0.1   
                                                       1              0.1   
                                                       2              0.1   
                                                       3              0.1   
                                                       4              0.1   

                                                              predict.param  \
idx name                                               q_idx                  
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0               0.95   
                                                       1               0.95   
                                                       2               0.95   
                                                       3               0.95   
                                                       4               0.95   

                                                              inf_time_base  \
idx name                                               q_idx                  
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0           0.008549   
                                                       1           0.007563   
                                                       2           0.008263   
                                                       3           0.008458   
                                                       4           0.009107   

                                                              inf_time_rel  \
idx name                                               q_idx                 
2   predict.algo=MI|predict.its=0.1|predict.param=0.95 0               1.0   
                                                       1               1.0   
        

In [18]:
df_lpt = build_df_lineplot(df_plt)
df_lpt.head()

,,perc_miss,inf_time,global_aligned_rank,ind_time,score_rel,score,aligned_rank,rank,global_dataset_aligned_rank
range_index,name,,,,,,,,,
0,predict.algo=MAFI|predict.its=0.1|predict.param=0.95,0.0,0.021514,20.5000,8.889205,1.000000,0.648898,2.5000,1.5000,10.5000
1,predict.algo=MAFI|predict.its=0.1|predict.param=0.95,10.0,0.019092,18.6000,8.889205,0.995961,0.646821,2.3000,1.4500,9.5500
2,predict.algo=MAFI|predict.its=0.1|predict.param=0.95,20.0,0.018508,36.6125,8.889205,0.907878,0.576979,2.3375,1.4375,18.7625
3,predict.algo=MAFI|predict.its=0.1|predict.param=0.95,40.0,0.018401,27.4875,8.889205,0.863944,0.542574,2.3625,1.4375,13.9125
4,predict.algo=MAFI|predict.its=0.1|predict.param=0.95,50.0,0.033418,19.5250,15.933035,0.941027,0.474241,2.4750,1.4750,19.5250


In [19]:
mem_usage(df_lpt)


    1.70 kiloB
    


In [20]:
df_cfg.dtypes

dataset              category
predict.algo         category
predict.its           float64
predict.param         float64
fit.sel.param           int64
fit.ind.max_depth       int64
fit.ind.type         category
fit.sel.its             int64
mod.type             category
mod.keyword          category
name                 category
dtype: object

## Plot Layout

In [21]:
graph_style = {'width':             '85%',
               'float':             'right',
               'z-index':            1,
               'position':           'relative',
               'margin-bottom':      '2cm'}

dropdown_menu_style = {'width':            '14%',
                       'backgroundColor':  'rgb(250, 250, 250)',
                       'float':            'left',
                       'z-index':            0,
                       'position':          'relative',
                       'border':            '1px solid gray'}

slider_menu_style = {'width':             '95%',
                     'backgroundColor':  'rgb(250, 250, 250)',
                     'float':            'left',
                     'margin-left':       '1cm',
                     'margin-bottom':     '2cm'}

extra_style = {'border':            '1px solid black'}

sep = html.Div(style={'clear': 'both'})

txt_box_style = {'width':             '25%',
                 'float':             'left',
                 'z-index':            1,
                 'position':           'relative',
                 'margin-left':       '2cm',
                 'border':            '1px solid gray'}

## Menus

### Dropdown

In [22]:
# Dropdown Menus
perf_dd_menus = generate_dropdown_menus_from_df(df_cfg, ignore_columns=['name'])

In [23]:
perf_dd_menus

[Label('dataset'),
 Dropdown(id='dataset', options=[{'label': 'msnbc', 'value': 'msnbc'}, {'label': 'nltcs', 'value': 'nltcs'}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=True),
 Label('predict-algo'),
 Dropdown(id='predict-algo', options=[{'label': 'MAFI', 'value': 'MAFI'}, {'label': 'MI', 'value': 'MI'}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=True),
 Label('predict-its'),
 Dropdown(id='predict-its', options=[{'label': '0.1', 'value': 0.1}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=True),
 Label('predict-param'),
 Dropdown(id='predict-param', options=[{'label': '0.95', 'value': 0.95}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=True),
 Label('fit-sel-param'),
 Dropdown(id='fit-sel-param', options=[{'label': '2', 'value': 2}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=True),
 Label('fit-ind-max_depth'),
 Dropdown(id='fit-ind-max_depth', options=[{'label': '4', 'value': 4}, {'label': 'IDC', 'value': 'IDC'}], value='IDC', multi=T

In [24]:
# Extra DropDown menus
targ_idx_dd = generate_dropdown_menus_from_df(df_qry, relevant_columns=["t_idx"])
show_data_dd = generate_dropdown_menu(
    "show_data", [False, True], labels=["No", "Yes"], default=0, multi=False
)

rank_score_dd = generate_dropdown_menu(
    "y_field",
    [
        "rank",
        "score",
        "score_rel",
        "aligned_rank",
        "global_aligned_rank",
        "global_dataset_aligned_rank",
    ],
    labels=[
        "Avg. Rank",
        "Avg. F1-Score",
        "Avg. Rel. F1-Score",
        "Aligned Rank",
        "Global Aligned Rank",
        "GDS Aligned Rank",
    ],
    default="score",
    multi=False,
)

perf_dd_menus.extend(targ_idx_dd)
perf_dd_menus.extend(show_data_dd)
perf_dd_menus.extend(rank_score_dd)

perf_dd_menus = html.Div(perf_dd_menus, style=dropdown_menu_style)

### Sliders

In [25]:
perf_sl_menus = generate_slider_menu("perc_miss")
perf_sl_menus.extend(generate_slider_menu("score_base", kind="range"))
perf_sl_menus.extend(generate_slider_menu("inf_time_rel", kind="log"))


perf_sl_menus = html.Div(perf_sl_menus, style=slider_menu_style)


disp_01 = html.Div(id="inf_time_rel-display", style={"margin-top": 20})
disp_02 = html.Div(id="score_base-display", style={"margin-top": 20})
disp_03 = html.Div(id="perc_miss-display", style={"margin-top": 20})

## Static App

In [26]:
perf_graph = dcc.Graph(id='lineplot', 
                       figure=generate_graph(df_lpt, kind='line', show_data=False))

stat_perf_contents = html.Div([perf_graph],
                              style=graph_style)

In [27]:
app = dash.Dash()
app.layout = html.Div(stat_perf_contents)

## Dynamic App

Re-uses some stuff from the static configuration.

### Initialization

In [28]:
dyn_perf_contents = [
    perf_dd_menus,
    stat_perf_contents,
    sep,
    perf_sl_menus,
    disp_01,
    disp_02,
    disp_03,
]

In [42]:
# Init App.
app = dash.Dash()
app.layout = html.Div(dyn_perf_contents)

In [45]:
menu_inputs, menu_names = extract_menu_inputs_menu_names_from_layout(app.layout)
# menu_names

### Callbacks for Text Updates.

This callback method needs to handle everything at once.

In [46]:
@app.callback(Output('inf_time_rel-display', 'children'),
              [Input('inf_time_rel', 'value')])
def display_value(value):
    msg = """
    Relative Inference Time \n
    Value = {:0.2f} \n
    (N.b. Linear slider value = {})
    """.format(10 ** value, value)
    
    return msg

@app.callback(Output('perc_miss-display', 'children'),
              [Input('perc_miss', 'value')])
def display_value(value):
    msg = """
    Percentage Missing Attributes \n
    Value = {} \n
    """.format(value)
    
    return msg


@app.callback(Output('score_base-display', 'children'),
              [Input('score_base', 'value')])
def display_value(value):
    msg = """
    Percentage Missing Attributes \n
    Value = {} \n
    """.format(value)
    
    return msg

### Plot Callback

This callback updates the entire plot.

In [47]:
df_plt.columns = [c.replace('.', '-') for c in df_plt.columns]
df_cfg.columns = [c.replace('.', '-') for c in df_cfg.columns]

In [48]:
kind='line' # Hardcoded here.


@app.callback(
    Output('lineplot', 'figure'),
    menu_inputs)
def update_lineplot(*args):
    
    menus = zip(menu_names, args)
    
    # Initializations
    filt_df = df_plt
    filt_df_params = df_cfg
    y_field = 'score'
    y_title = 'Average '+ str(y_field)
    
    for name, values in menus:

        if name in {'t_idx'}:
            filt_df = filter_dataframe(filt_df, name, values)
        elif name in {'perc_miss'}:
            filt_df = filter_dataframe(filt_df, name, values, kind='Slider')
        elif name in {'inf_time_rel'}:
            print("Values: inf_time_rel: {}".format(values))
            values = 10**values
            filt_df = filter_dataframe(filt_df, name, values, kind='timeout')
        elif name in {'score_base'}:
            print("Values: perc_miss: {}".format(values))
            filt_df = filter_dataframe(filt_df, name, values, kind='RangeSlider')
        elif name in {'show_data'}:
            show_data = values
            #print("show_data value: {}".format(values))
        elif name in {'baseline'}:
            baseline_name = values
        elif name in {'y_field'}:
            y_field = values
            y_title = 'Average '+ str(y_field)
        else:
            filt_df_params = filter_dataframe(filt_df_params, name, values)
    
    try:
        #print(filt_df.head())
        filt_df = filt_df.loc[filt_df_params.index.values]  # Only keep the entries with indices present in df_params
        plot_df = build_df_lineplot(filt_df)
        
    except ValueError as e: 
        msg = "Caught ValueError, this -sometimes- happens whenever no data is present in the plot: {}".format(e)
        print(msg)
        plot_df = filt_df
        pass

    figure_parameters = generate_graph(plot_df,
                                       kind=kind,
                                       show_data=show_data,
                                       y_title=y_title,
                                       x_title='Missing Attributes (%)',
                                       y_field=y_field)
    
    return figure_parameters

## Run App

Run the actual browser applet.

In [49]:
app.run_server(port=8881)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8881/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1562757267 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1562757267 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1562757267 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1562757268 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1562757267 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:49] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1562757267 HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 1

Values: perc_miss: [0, 100]
Values: inf_time_rel: 2


127.0.0.1 - - [10/Jul/2019 15:55:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:55:53] "POST /_dash-update-component HTTP/1.1" 200 -


Values: perc_miss: [0, 100]
Values: inf_time_rel: 2
Values: perc_miss: [0, 100]
Values: inf_time_rel: 2


...

Values: perc_miss: [0, 100]
Values: inf_time_rel: 2


127.0.0.1 - - [10/Jul/2019 15:56:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 15:56:27] "POST /_dash-update-component HTTP/1.1" 200 -


Values: perc_miss: [0, 100]
Values: inf_time_rel: 7


127.0.0.1 - - [10/Jul/2019 16:01:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2019 16:01:03] "POST /_dash-update-component HTTP/1.1" 200 -


Values: perc_miss: [0, 100]
Values: inf_time_rel: 7
